In [1]:
from binance import ThreadedWebsocketManager
from binance.client import Client
import pandas as pd
import numpy as np
import datetime

In [5]:
api_key = "DAX5KWi4IudakbEVvmQsjgqRYkSVycWQralfOuIWr7xSyQ4chqn192dLS4QGkG1q"  # TESTING testnet.binance.vision
api_secret = "i5YaSzFjlZTkrqsdMzEGUoukzkKehozhtNi0E5XPVP2UnbzyDuFB8RJuN1JMSTKZ"  # TESTING testnet.binance.vision

In [6]:
client = Client(api_key=api_key, api_secret=api_secret, testnet=True)
client.get_account()

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'brokered': False,
 'updateTime': 1667394968422,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '1.00000000', 'locked': '0.00000000'},
  {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': ['SPOT']}

In [25]:
class LongOnlyTrader():
    
    def __init__(self, symbol, interval, units=0.1, position=0, sma1=7, sma2=20):
        self.symbol = symbol
        self.interval = interval
        self.units = units
        self.position = position
        self.available_intervals = ["1m", "3m", "5m", "15m", "30m", "1h", "2h", "4h", "6h", "8h", "12h", "1d", "3d", "1w", "1M"]

#         Strategy attributs
        self.sma1 = sma1
        self.sma2 = sma2
        
    def start_trading(self, past_mins=25):
        self.twm = ThreadedWebsocketManager()
        self.twm.start()
        
        if self.interval in self.available_intervals:
            self.get_recent_prices(past_mins)
            self.twm.start_kline_socket(callback=self.stream_candles,
                                        symbol=self.symbol, interval=self.interval)
        # "else" to be added later in the course 
    
    def get_recent_prices(self, past_mins):
        now = datetime.datetime.utcnow()
        past = str(now - datetime.timedelta(minutes=past_mins))
        
        bars = client.get_historical_klines(self.symbol, self.interval, past)
        
        df = pd.DataFrame(bars)
        df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms")
        df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume",
                      "Clos Time", "Quote Asset Volume", "Number of Trades",
                      "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
        df = df[["Date", "Open", "High", "Low", "Close", "Volume"]].copy()
        df.set_index("Date", inplace=True)
        df["sma1"] = df.Close.rolling(self.sma1).mean()
        df["sma2"] = df.Close.rolling(self.sma2).mean()
        for column in df.columns:
            df[column] = pd.to_numeric(df[column], errors = "coerce")
        df["Complete"] = [True for row in range(len(df)-1)] + [False]
        
        self.data = df

    def stream_candles(self, msg):
        
        # extract the required items from msg
        event_time = pd.to_datetime(msg["E"], unit = "ms")
        start_time = pd.to_datetime(msg["k"]["t"], unit = "ms")
        first = float(msg["k"]["o"])
        high = float(msg["k"]["h"])
        low = float(msg["k"]["l"])
        close = float(msg["k"]["c"])
        volume = float(msg["k"]["v"])
        complete= msg["k"]["x"]
        # print out
        print(".", end="", flush=True)
    
        # feed df (add new bar / update latest bar)
        self.data.loc[start_time] = [first, high, low, close, volume, complete, np.nan, np.nan]
        self.data["sma1"] = self.data.Close.rolling(self.sma1).mean()
        self.data["sma2"] = self.data.Close.rolling(self.sma2).mean()
        if complete:
            self.define_strategy()
            self.execute_order()

    def define_strategy(self):
        df = self.data.copy()
        # Define strategy  --  If last candle is green, buy. If red, sell
        cond = df.sma1 > df.sma2  # Red candle
        
        df["Position"] = 0
        df.loc[cond, "Position"] = 1
        
        self.prepare_data = df.copy()
        
    def execute_order(self):
        if self.prepare_data["Position"].iloc[-1] == 1:  # Buy
            if self.position == 0:
                order = client.create_order(symbol=self.symbol, side="BUY", type="MARKET", quantity=self.units)
                self.position = 1
                print("BUYING BITCOIN")

        if self.prepare_data["Position"].iloc[-1] == 0: # Sell:
            if self.position == 1:
                order = client.create_order(symbol=self.symbol, side="SELL", type="MARKET", quantity=self.units)
                self.position = 0
                print("SELLING BITCOIN")

In [26]:
trader = LongOnlyTrader("BTCUSDT", "1m", 0.5)

In [27]:
trader.start_trading()

....

In [31]:
trader.prepare_data

,Open,High,Low,Close,Volume,sma1,sma2,Complete,Position
Date,,,,,,,,,
2022-11-24 14:03:00,16571.86,16572.17,16561.40,16566.62,6.446833,NaN,NaN,True,0
2022-11-24 14:04:00,16566.62,16570.90,16565.96,16566.06,6.529141,NaN,NaN,True,0
2022-11-24 14:05:00,16566.08,16571.28,16565.34,16567.08,6.694864,NaN,NaN,True,0
2022-11-24 14:06:00,16567.08,16567.08,16560.02,16563.38,6.008125,NaN,NaN,True,0
2022-11-24 14:07:00,16563.65,16564.53,16561.53,16562.85,6.212619,NaN,NaN,True,0
2022-11-24 14:08:00,16562.32,16570.28,16561.30,16566.35,8.946429,NaN,NaN,True,0
2022-11-24 14:09:00,16565.66,16570.16,16565.66,16570.16,5.635924,16566.071429,NaN,True,0
2022-11-24 14:10:00,16569.88,16571.01,16568.76,16570.83,3.972438,16566.672857,NaN,True,0
2022-11-24 14:11:00,16570.68,16573.66,16567.75,16567.75,6.965654,16566.914286,NaN,True,0


....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................SELLING BITCOIN
..............................................................................................................................................................................................................................................................................................................................................................................................BUYING BITCOIN
....

In [38]:
client.get_account()

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'brokered': False,
 'updateTime': 1669332961160,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '1.00000000', 'locked': '0.00000000'},
  {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '10013.46266488', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': ['SPOT']}

In [37]:
trader.twm.stop()

.

CANCEL read_loop
